# Master Stock Analysis Summary

This notebook provides a summary of volume analysis and signals for all tracked stocks.

In [ ]:
# Install required packages
!pip install pandas numpy matplotlib seaborn google-cloud-bigquery pandas-gbq plotly

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from google.cloud import bigquery
from datetime import datetime, timedelta
from IPython.display import HTML, display

# Set plot style
sns.set(style="darkgrid")
plt.rcParams['figure.figsize'] = (14, 8)

In [ ]:
# Authenticate with Google Cloud
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Configuration
PROJECT_ID = "{PROJECT_ID}"  # Will be replaced with actual project ID
DATASET = "trading_insights"
TRACKED_STOCKS = {TRACKED_STOCKS_JSON}  # Will be replaced with actual tracked stocks

In [ ]:
# Initialize BigQuery client
client = bigquery.Client(project=PROJECT_ID)

## Fetch Latest Analysis Results

Let's fetch the latest analysis results for all tracked stocks from BigQuery.

In [ ]:
# Fetch latest analysis results from master summary table
query = f"""
WITH LatestAnalysis AS (
  SELECT
    ticker,
    MAX(timestamp) AS latest_timestamp
  FROM
    `{PROJECT_ID}.{DATASET}.master_summary`
  GROUP BY
    ticker
)
SELECT
  s.*
FROM
  `{PROJECT_ID}.{DATASET}.master_summary` s
JOIN
  LatestAnalysis l
ON
  s.ticker = l.ticker
  AND s.timestamp = l.latest_timestamp
ORDER BY
  s.confidence DESC
"""

try:
    summary_df = client.query(query).to_dataframe()
    print(f"Fetched analysis results for {len(summary_df)} stocks")
except Exception as e:
    print(f"Error fetching analysis results: {str(e)}")
    # Create empty dataframe with expected columns
    summary_df = pd.DataFrame(columns=[
        'date', 'ticker', 'direction', 'signal', 'confidence',
        'volume_score', 'is_spike', 'spike_strength', 'notes', 'notebook_url', 'timestamp'
    ])

# If no data is available, create sample data for demonstration
if summary_df.empty:
    print("No data available in BigQuery. Creating sample data for demonstration.")
    
    # Create sample data
    sample_data = []
    
    # Add buy stocks
    for ticker in TRACKED_STOCKS['buy']:
        sample_data.append({
            'date': datetime.now().date(),
            'ticker': ticker,
            'direction': 'buy',
            'signal': np.random.choice(['STRONG_BUY', 'POTENTIAL_REVERSAL', 'NEUTRAL'], p=[0.3, 0.3, 0.4]),
            'confidence': np.random.uniform(0.5, 1.0),
            'volume_score': np.random.uniform(1.0, 3.0),
            'is_spike': np.random.choice([True, False], p=[0.3, 0.7]),
            'spike_strength': np.random.uniform(1.0, 2.0),
            'notes': f"Sample data for {ticker}",
            'notebook_url': f"https://colab.research.google.com/drive/your-notebook-id-for-{ticker}",
            'timestamp': datetime.now()
        })
    
    # Add short stocks
    for ticker in TRACKED_STOCKS['short']:
        sample_data.append({
            'date': datetime.now().date(),
            'ticker': ticker,
            'direction': 'short',
            'signal': np.random.choice(['STRONG_SHORT', 'POTENTIAL_REVERSAL', 'NEUTRAL'], p=[0.3, 0.3, 0.4]),
            'confidence': np.random.uniform(0.5, 1.0),
            'volume_score': np.random.uniform(1.0, 3.0),
            'is_spike': np.random.choice([True, False], p=[0.3, 0.7]),
            'spike_strength': np.random.uniform(1.0, 2.0),
            'notes': f"Sample data for {ticker}",
            'notebook_url': f"https://colab.research.google.com/drive/your-notebook-id-for-{ticker}",
            'timestamp': datetime.now()
        })
    
    summary_df = pd.DataFrame(sample_data)

# Display summary dataframe
summary_df.head()

## Dashboard: Signal Summary

Let's create a dashboard to visualize the signals for all tracked stocks.

In [ ]:
# Create a function to generate a styled HTML table
def generate_styled_table(df, title):
    # Create a copy of the dataframe with selected columns
    table_df = df[['ticker', 'direction', 'signal', 'confidence', 'volume_score', 'is_spike']].copy()
    
    # Add hyperlinks to ticker symbols
    table_df['ticker'] = table_df.apply(
        lambda row: f"<a href='{df.loc[row.name, 'notebook_url']}' target='_blank'>{row['ticker']}</a>",
        axis=1
    )
    
    # Format confidence as percentage
    table_df['confidence'] = table_df['confidence'].apply(lambda x: f"{x:.0%}")
    
    # Format volume score
    table_df['volume_score'] = table_df['volume_score'].apply(lambda x: f"{x:.2f}")
    
    # Format is_spike as Yes/No
    table_df['is_spike'] = table_df['is_spike'].apply(lambda x: "Yes" if x else "No")
    
    # Rename columns
    table_df.columns = ['Ticker', 'Direction', 'Signal', 'Confidence', 'Volume Z-Score', 'Volume Spike']
    
    # Convert to HTML
    html = f"<h2>{title}</h2>"
    html += "<style>"
    html += "table { border-collapse: collapse; width: 100%; }"
    html += "th, td { padding: 8px; text-align: left; border-bottom: 1px solid #ddd; }"
    html += "th { background-color: #f2f2f2; }"
    html += "tr:hover { background-color: #f5f5f5; }"
    html += "a { text-decoration: none; color: #0366d6; }"
    html += "a:hover { text-decoration: underline; }"
    html += "</style>"
    html += table_df.to_html(escape=False, index=False)
    
    return html

# Split dataframe by direction
buy_df = summary_df[summary_df['direction'] == 'buy'].sort_values('confidence', ascending=False)
short_df = summary_df[summary_df['direction'] == 'short'].sort_values('confidence', ascending=False)

# Generate styled tables
buy_table = generate_styled_table(buy_df, "Buy Candidates")
short_table = generate_styled_table(short_df, "Short Candidates")

# Display tables
display(HTML(buy_table))
display(HTML(short_table))

## Volume Analysis Visualization

Let's visualize the volume analysis results for all tracked stocks.

In [ ]:
# Create a scatter plot of volume score vs. confidence
fig = px.scatter(
    summary_df,
    x="volume_score",
    y="confidence",
    color="direction",
    size="spike_strength",
    hover_name="ticker",
    hover_data=["signal", "notes"],
    title="Volume Score vs. Confidence",
    labels={
        "volume_score": "Volume Z-Score",
        "confidence": "Signal Confidence",
        "direction": "Direction",
        "spike_strength": "Spike Strength"
    },
    color_discrete_map={"buy": "green", "short": "red"}
)

# Add ticker labels
fig.update_traces(textposition='top center')
fig.update_layout(
    height=600,
    width=900,
    xaxis=dict(title="Volume Z-Score", range=[0, max(summary_df['volume_score']) * 1.1]),
    yaxis=dict(title="Signal Confidence", range=[0, 1.1])
)

fig.show()

## Signal Distribution

Let's visualize the distribution of signals across all tracked stocks.

In [ ]:
# Count signals by type and direction
signal_counts = summary_df.groupby(['direction', 'signal']).size().reset_index(name='count')

# Create a grouped bar chart
fig = px.bar(
    signal_counts,
    x="signal",
    y="count",
    color="direction",
    title="Signal Distribution",
    labels={
        "signal": "Signal Type",
        "count": "Count",
        "direction": "Direction"
    },
    color_discrete_map={"buy": "green", "short": "red"}
)

fig.update_layout(
    height=500,
    width=800,
    xaxis=dict(title="Signal Type"),
    yaxis=dict(title="Count")
)

fig.show()

## Top Signals

Let's identify the top signals for both buy and short directions.

In [ ]:
# Function to display top signals
def display_top_signals(df, direction, n=3):
    # Filter by direction and non-neutral signals
    filtered_df = df[(df['direction'] == direction) & (df['signal'] != 'NEUTRAL')]
    
    # Sort by confidence
    top_df = filtered_df.sort_values('confidence', ascending=False).head(n)
    
    if len(top_df) == 0:
        print(f"No {direction} signals found")
        return
    
    print(f"Top {n} {direction.capitalize()} Signals:")
    for i, (_, row) in enumerate(top_df.iterrows(), 1):
        print(f"{i}. {row['ticker']} - {row['signal']} (Confidence: {row['confidence']:.0%})")
        print(f"   Volume Z-Score: {row['volume_score']:.2f}")
        print(f"   Notes: {row['notes']}")
        print(f"   Notebook: {row['notebook_url']}")
        print()

# Display top buy signals
display_top_signals(summary_df, 'buy')

# Display top short signals
display_top_signals(summary_df, 'short')

## Summary and Next Steps

This notebook provides a summary of volume analysis and signals for all tracked stocks.

### Key Findings:
- Number of buy signals: {BUY_SIGNAL_COUNT}
- Number of short signals: {SHORT_SIGNAL_COUNT}
- Top buy candidate: {TOP_BUY_TICKER}
- Top short candidate: {TOP_SHORT_TICKER}

### Next Steps:
1. Review individual notebooks for stocks with strong signals
2. Monitor volume patterns for potential trade opportunities
3. Update analysis daily to track changes in volume patterns